In [1]:
import os
import sys
sys.path.append("../")
import json
import pickle
import random
from sim.sim_use import make_random_paire_list_instr
from exploration.history import History
import numpy as np
from exploration.random.func import RANDOM
from exploration.env.func import Env
from  exploration.imgep.OptimizationPolicy import OptimizationPolicykNN


In [2]:
class EfficientOnlineIntervalArea:
    def __init__(self, radius):
        self.radius = radius
        self.intervals = []  # List of (start, end) tuples, maintained sorted
        self.total_area = 0.0
    
    def add_ball(self, center):
        """Add a new ball using binary search for efficient insertion"""
        new_start = center - self.radius
        new_end = center + self.radius
        
        # Find insertion position using binary search
        left_idx = self._find_left_index(new_start)
        right_idx = self._find_right_index(new_end)
        
        if left_idx > right_idx:
            # No overlap, insert new interval
            self.intervals.insert(left_idx, (new_start, new_end))
            self.total_area += new_end - new_start
            return self.total_area
        
        # Merge overlapping intervals
        merged_start = min(new_start, self.intervals[left_idx][0])
        merged_end = max(new_end, self.intervals[right_idx][1])
        
        # Calculate area to remove
        areas_to_remove = 0
        for i in range(left_idx, right_idx + 1):
            areas_to_remove += self.intervals[i][1] - self.intervals[i][0]
        
        # Replace overlapping intervals with merged one
        del self.intervals[left_idx:right_idx + 1]
        self.intervals.insert(left_idx, (merged_start, merged_end))
        
        # Update total area
        self.total_area = self.total_area - areas_to_remove + (merged_end - merged_start)
        
        return self.total_area
    
    def _find_left_index(self, start):
        """Find the first interval that could overlap with start"""
        left, right = 0, len(self.intervals)
        while left < right:
            mid = (left + right) // 2
            if self.intervals[mid][1] < start:
                left = mid + 1
            else:
                right = mid
        return left
    
    def _find_right_index(self, end):
        """Find the last interval that could overlap with end"""
        left, right = 0, len(self.intervals)
        while left < right:
            mid = (left + right) // 2
            if self.intervals[mid][0] <= end:
                left = mid + 1
            else:
                right = mid
        return left - 1
    
    def get_total_area(self):
        return self.total_area
    
    def get_interval_count(self):
        return len(self.intervals)

# Example usage
efficient_online = EfficientOnlineIntervalArea(radius=1.0)

centers = [1.0, 2.5, 4.0, 3.2, 6.0, 2.8]
for center in centers:
    area = efficient_online.add_ball(center)
    print(f"Added {center}: area={area:.2f}, intervals={efficient_online.get_interval_count()}")

print(f"\nFinal: {efficient_online.get_total_area():.2f}")

Added 1.0: area=2.00, intervals=1
Added 2.5: area=3.50, intervals=1
Added 4.0: area=5.00, intervals=1
Added 3.2: area=5.00, intervals=1
Added 6.0: area=7.00, intervals=1
Added 2.8: area=7.00, intervals=1

Final: 7.00


In [3]:
class GoalGenerator():
    def __init__(self,
                 num_bank:int,
                 ):
        super().__init__()
        self.num_bank = num_bank
        self.k_time = 0
        self.k_miss = 0
    def __call__(self,coords:np.ndarray)->np.ndarray:
        min_ = coords.min(axis=-1)
        max_ = coords.max(axis=-1)
        out = np.random.uniform((1-np.sign(min_)*0.6)*min_,4.0*max_)
        return out

In [4]:
class History_(History):
    def __init__(self,max_size=100):
        super(History_,self).__init__(max_size)
        self.memory_reward = []
    def store(self,sample:dict[list]):
        super().store(sample)
        self.memory_reward.append(sample['reward'])

In [5]:
class Env_observation2_subset:
    def __init__(self,pp:list[str]):
        self.pp = pp
    def __call__(self,output):
        out_list = []
        for o in self.pp:
                out_list.append(np.array(output[o]).flatten())
        return np.concatenate(out_list)

In [6]:
class OptimizationPolicykNN_(OptimizationPolicykNN):
    def __init__(self,
                k=4,
                mutation_rate = 0.1,
                max_len=50,
                num_addr = 20,
                num_bank = 4,
                min_instr = 5,
                max_instr = 50):
                super(OptimizationPolicykNN_,self).__init__(
                k=4,
                mutation_rate = 0.1,
                max_len=50,
                num_addr = 20,
                num_bank = 4,
                min_instr = 5,
                max_instr = 50)
    def select_closest_codes(self,H,coords:np.ndarray,goal:np.ndarray):
        idx = self.feature2closest_code(coords.reshape((1,-1)),goal)
        output = {"program": {"core0":[],"core1":[]},}
        for id_ in idx:
            output["program"]["core0"].append(H.memory_program["core0"][id_])
            output["program"]["core1"].append(H.memory_program["core1"][id_])
        return output
    def __call__(self,goal:np.ndarray,H:History,coords:np.ndarray)->dict:
        closest_codes = self.select_closest_codes(H,coords,goal) #most promising sample from the history
        output = self.mix(closest_codes) #expansion strategie: small random mutation
        return output
class Normalize:
    """Affine normalization
    """
    def __init__(self):
        self.min_=None
        self.max_= None
        self.g = 1
    def fit(self,x):
        self.min_ = x.min(axis=0)
        self.max_ = x.max(axis=0)
        self.g=0
    def transform(self,x):
        if self.g: 
            raise TypeError(f"User must call method Normalize.fit before calling method Normalize.transform")
        return (x - self.min_)/(self.max_-self.min_)
class IMGEP_:
    """
    N: int. The experimental budget
    N_init: int. Number of experiments at random
    H: History. Buffer containing codes and signature pairs
    G: GoalGenerator.
    Pi: OptimizationPolicy.
    """
    def __init__(self,
                 N:int,
                 N_init:int,
                 E:Env,
                 H:History,
                 G:GoalGenerator,
                 Pi:OptimizationPolicykNN,
                 periode:int = 1,
                 max_len:int = 100):
        self.N = N
        self.env = E
        self.H = H
        self.G = G
        self.N_init = N_init
        self.Pi = Pi
        self.periode = periode
        self.max_len = max_len
        self.start = 0
        self.periode_expl = 10
        self.k = 0
    def take(self,sample:dict,N_init:int):
        """Takes the ``N_init`` first steps from the ``sample`` dictionnary to initialize the exploration.
        Then the iterator i is set to N_init directly
        """
        print("sampl", sample.keys())
        for key in sample["memory_perf"].keys():
            self.H.memory_perf[key]= list(sample["memory_perf"][key][:N_init])
        self.H.memory_program["core0"] = sample["memory_program"]["core0"][:N_init]
        self.H.memory_program["core1"] = sample["memory_program"]["core1"][:N_init]
        self.start = N_init
        for j in range(len(H)):
            
            dd = {key:sample["memory_perf"][key][j] for key in sample["memory_perf"].keys() if key in pp}
            H.memory_tab.append(env_observation2_subset(dd))


class IMGEP(IMGEP_):
    def __init__(self,
                 N:int,
                 N_init:int,
                 E:Env,
                 H:History,
                 G:GoalGenerator, 
                 Pi:OptimizationPolicykNN_,
                 Norm:Normalize,
                 periode:int = 1,
                 max_len:int = 100,
                 tol = 1e-6):
        super(IMGEP,self).__init__(N,N_init, En,H,G,Pi, periode = periode, max_len = max_len)
        self.norm = Norm
        self.tol = tol
        self.efficient_online = [EfficientOnlineIntervalArea(radius=0.001) for j in range(88)]
        self.total_area = 0
    def variation_diversity(self,T):
        out = 0
        for j in range(len(T)):
            d = self.efficient_online[j].add_ball(T[j])
            out +=d
        delta = out - self.total_area 
        self.total_area = out
        return delta
    def _SVD(self):
        in_ = np.array(np.array(self.H.memory_tab))
        self.norm.fit(in_)
        in_0 = self.norm.transform(in_)
        U,sigma,Vh =np.linalg.svd(in_0)
        return U,sigma,Vh,in_0
    def reward(self,coords):
        reward = self.variation_diversity(coords)
        return reward
    def __call__(self):
        """Performs the exploration.
        """
        for i in range(self.start,self.N+1):
            if i%100==0:
                print(f"{i} iterations")
            if i<self.N_init:
                parameter = make_random_paire_list_instr(self.max_len,num_addr=self.env.num_addr)
            else:
                if (i-self.N_init)%(self.periode*10)==0:
                    U,sigma,Vh,in_0 = self._SVD()
                if (i-self.N_init)%self.periode==0:
                    idx_axis = np.random.randint(0,sum(sigma>self.tol))
                if i ==self.N_init:
                    in_ = np.array(self.H.memory_tab)
                    coords = self.norm.transform(in_)@Vh.transpose()
                goal = self.G(coords[:,idx_axis])
                parameter = self.Pi(goal,self.H, coords[:,idx_axis])
            observation = self.env(parameter)
            subset = env_observation2_subset(observation)
            self.H.memory_tab.append(subset)
            if i >self.N_init:
                in_ = np.array(self.H.memory_tab)
                coords = self.norm.transform(in_)@Vh.transpose()
            reward = self.reward(subset)
            print("reward",i, reward)
            self.H.store({"program":parameter}|observation|{"reward":reward})

In [7]:
num_bank = 4
num_addr = 20

pp = ['shared_cache_miss',
      'general_shared_cache_miss',
      'general_shared_cache_miss_core0',
      'general_shared_cache_miss_core1',
      'miss_ratios',
      'miss_ratios_global',
      'miss_ratios_global0',
      'miss_ratios_global1',
      'miss_ratios_core0',
      'miss_ratios_core1',
      'time_core0_together',
      'time_core1_together',
      'time_core0_alone',
      'time_core1_alone',
      'miss_count',
      'miss_count_core0',
      'miss_count_core1',
      'diff_ratios_core0',
      'diff_ratios_core1',
      'diff_time0',
      'diff_time1',
      'miss_ratios_detailled',
      'miss_ratios_core0_detailled',
      'miss_ratios_core1_detailled',
    ]
periode  = 20
max_len = 50
num_banks = 4
num_addr = 20
mutation_rate = 0.1
N = 1200
N_init = 1000
k = 2
min_len=5

In [8]:
env_observation2_subset = Env_observation2_subset(pp)

In [9]:
En = Env(repetition=1,num_banks = num_bank,num_addr = num_addr)
H = History_(N)
Pi = OptimizationPolicykNN_(k=k,mutation_rate=mutation_rate,max_len=max_len,num_addr=num_addr,num_bank=num_bank,min_instr=min_len,max_instr=max_len)
G = GoalGenerator(num_banks)
Norm = Normalize()

folder = "all_data/svd_results"

In [10]:
imgep = IMGEP(N,N_init, En,H,G,Pi,Norm, periode = periode, max_len = max_len)

# Performing random exploration

In [11]:
while True:
    print("opening data")
    try:
        with open(f"{folder}/history_rand_N_{N}_0.pkl","rb") as f:
            sample_rand = pickle.load(f)
            content_random = sample_rand["memory_perf"]
        print("data opened")
        break
    except:
        print("start random exploration")
        H_rand = History(max_size=N)
        H2_rand = History(max_size=N)
        rand = RANDOM(N = N,E = En, H = H_rand, H2 = H2_rand,max_=max_len)
        rand()
        H2_rand.save_pickle(f"{folder}/history_rand_N_{N}")

opening data
data opened


In [12]:
content_random.keys()

dict_keys(['shared_cache_miss', 'general_shared_cache_miss', 'general_shared_cache_miss_core0', 'general_shared_cache_miss_core1', 'miss_ratios', 'miss_ratios_global', 'miss_ratios_global0', 'miss_ratios_global1', 'miss_ratios_core0', 'miss_ratios_core1', 'time_core0_together', 'time_core1_together', 'time_core0_alone', 'time_core1_alone', 'miss_count', 'miss_count_core0', 'miss_count_core1', 'diff_ratios_core0', 'diff_ratios_core1', 'diff_time0', 'diff_time1', 'miss_ratios_detailled', 'miss_ratios_core0_detailled', 'miss_ratios_core1_detailled', 'core1_L1_cache_miss', 'core1_L2_cache_miss', 'core1_L3_cache_miss', 'core0_L1_cache_miss', 'core0_L2_cache_miss', 'core0_L3_cache_miss'])

# Performing IMGEP

In [13]:
imgep.take(sample_rand,N_init)

sampl dict_keys(['memory_perf', 'memory_program'])


In [14]:
imgep()
folder = "all_data/svd_results"
H.save_pickle(f"{folder}/history_kNN_{k}_N_{N}_svd")

1000 iterations
reward 1000 0.17600000000003102
reward 1001 0.11800000000003552
reward 1002 0.10600000000003673
reward 1003 0.08000000000002555
reward 1004 0.07200000000003509
reward 1005 0.038000000000026235
reward 1006 0.01800000000001445
reward 1007 0.060000000000028364
reward 1008 0.006000000000021544
reward 1009 0.0
reward 1010 0.026000000000033885
reward 1011 0.0259999999999746
reward 1012 0.034000000000021346
reward 1013 0.017326259946978007
reward 1014 0.05600000000003447
reward 1015 0.004000000000011994
reward 1016 0.016000000000021553
reward 1017 0.0040000000000048885
reward 1018 0.008000000000000007
reward 1019 0.010000000000004894
reward 1020 0.03600000000001935
reward 1021 0.016000000000038206
reward 1022 0.028000000000040437
reward 1023 0.02200000000001423
reward 1024 0.03999999999999093
reward 1025 0.03599999999992298
reward 1026 0.03200000000003844
reward 1027 0.010000000000018883
reward 1028 0.020000000000033324
reward 1029 0.012000000000040423
reward 1030 0.0040000000

In [24]:
from sortedcontainers import SortedList  # pip install sortedcontainers


def compute_area_interval_tree(centers, radius):
    """
    Efficient approach for very large datasets using interval trees
    """
    if not centers:
        return 0.0
    
    # Create and sort intervals
    intervals = [(c - radius, c + radius) for c in centers]
    intervals.sort()
    
    # Use a sorted list to efficiently merge intervals
    merged = SortedList()
    
    for start, end in intervals:
        # Find overlapping intervals
        idx = merged.bisect_right((start, float('inf'))) - 1
        
        if idx >= 0 and merged[idx][1] >= start:
            # Merge with existing interval
            old_start, old_end = merged.pop(idx)
            new_start = min(old_start, start)
            new_end = max(old_end, end)
            merged.add((new_start, new_end))
        else:
            # Add as new interval
            merged.add((start, end))
    
    # Calculate total length
    total_length = sum(end - start for start, end in merged)
    
    return total_length

In [25]:
folder = ["all_data/svd_results","all_data/svd_results","all_data/data_weak"]
image_folder ="all_images/svd"
N = int(10000)
N_init = 1000
ks = [2]
ks2 = [1,2,3,5,10]
ks2 = []
num_bank = 4
num_addr = 20
file_mix = lambda k,N: f"history_weak_{k}_N_{N}_0.pkl"
file_imgep_ir = lambda k,N: f"history_kNN_{k}_N_{N}_lp_0.pkl"
file_imgep_svd = lambda k,N: f"history_kNN_{k}_N_{N}_svd_7.pkl"
file_imgep_svd2 = lambda k,N: f"history_kNN_{k}_N_{N}_svd_1.pkl"
files = []
files +=[{"folder":folder[1],"file":file_imgep_svd(k,N),"name":f"imgep_svd k={k},N={N}","k":k,"N":N,"type":"imgep_svd"} for k in ks]
files +=[{"folder":folder[1],"file":file_imgep_svd2(k,N),"name":f"imgep_svd k={k},N={N}","k":k,"N":N,"type":"imgep_svd"} for k in ks2]

random = {"folder":folder[0],
            "N":N,
            "file":f"history_rand_N_{N}_0.pkl",
            "name": "random"}
config = {"files":files,
          "N_init":N_init,
          "N":N,
          "image_folder":image_folder,
          "random":random,
          "num_bank": num_bank,
          "num_addr":num_addr,
          "ks":ks}


In [26]:

import pickle
from visualisation.visu import diversity_time_iteration2
from visualisation.visu2 import comparaison3, comparaison_ratios_iterations, comparaison_ratios_global_iterations
from exploration.imgep.intrinsic_reward import IR
import json
import sys
import os
from visualisation.visu_modules import visu_modules

N = config["N"]
N_init = config["N_init"]
image_folder = config["image_folder"]
files = config["files"]
random = config["random"]
num_bank = config["num_bank"]
num_addr = config["num_addr"]
ks = config["ks"]
with open(os.path.join(random["folder"],random["file"]),"rb") as f:
    sample = pickle.load(f)
content_random = sample["memory_perf"]


contents_ = []
for data in files:
    with open(os.path.join(data["folder"],data["file"]), "rb") as f:
        sample = pickle.load(f)
        content = sample["memory_perf"]
        contents_.append((data["name"],content,data["k"]))
    comparaison3(content_random, 
                 content, 
                 name = [f"{image_folder}/{nn}_{data['type']}_{data['k']}_{N}" for nn in ["ratios","time"]], 
                 title=[f"{name} k = {data['k']}, {data['N']} iterations" for name in ["miss ratios", "time"]],num_bank=num_bank, num_row = num_addr//16 + 1)
for k_ in ks:
    diversity_time_iteration2(content_random,
                        [(data["name"],data["k"],f"{data['folder']}/{data['file']}") for data in files if data["k"]==k_],
                       title = f"comparaison_time_diversity_{k_}_{N}",
                       folder = image_folder)
comparaison_ratios_iterations([(a[0],a[1]) for a in contents_] + [("random", content_random)],
                name = f"{image_folder}/comp_ratios_iteration_{N}", k = k_)
comparaison_ratios_global_iterations([(a[0],a[1]) for a in contents_] + [("random", content_random)],
                                            name = f"{image_folder}/comp_global_ratios_iteration_{N}", k = k_)


together 10001
0 alone 10001
name all_images/svd/comp_global_ratios_iteration_10000
together 10000
0 alone 10000
name all_images/svd/comp_global_ratios_iteration_10000


<Figure size 640x480 with 0 Axes>

In [25]:
import matplotlib.pyplot as plt

In [42]:
sum(content_random['miss_ratios_global']==-1)

0

In [45]:
for key in content_random:
    if key not in ['shared_cache_miss', 'general_shared_cache_miss', 'general_shared_cache_miss_core0', 'general_shared_cache_miss_core1']:
        min_ = content_random[key].min()
        max_ = content_random[key].max()
        cont = (content_random[key] - min_)/(max_-min_)
        print(key,cont.shape)

miss_ratios (10000, 4)
miss_ratios_global (10000,)
miss_ratios_global0 (10000,)
miss_ratios_global1 (10000,)
miss_ratios_core0 (10000, 4)
miss_ratios_core1 (10000, 4)
time_core0_together (10000,)
time_core1_together (10000,)
time_core0_alone (10000,)
time_core1_alone (10000,)
miss_count (10000, 4)
miss_count_core0 (10000, 4)
miss_count_core1 (10000, 4)
diff_ratios_core0 (10000, 4)
diff_ratios_core1 (10000, 4)
diff_time0 (10000,)
diff_time1 (10000,)
miss_ratios_detailled (10000, 2, 4)
miss_ratios_core0_detailled (10000, 2, 4)
miss_ratios_core1_detailled (10000, 2, 4)
core1_L1_cache_miss (10000, 20)
core1_L2_cache_miss (10000, 20)
core1_L3_cache_miss (10000, 20)
core0_L1_cache_miss (10000, 20)
core0_L2_cache_miss (10000, 20)
core0_L3_cache_miss (10000, 20)


In [50]:
sum(content_random['core0_L3_cache_miss']==-1)

array([8207, 8195, 8133, 8240, 8169, 8188, 8150, 8208, 8214, 8234, 8179,
       8278, 8247, 8284, 8187, 8203, 8212, 8187, 8231, 8237])

In [53]:
p = ['shared_cache_miss', 'general_shared_cache_miss', 'general_shared_cache_miss_core0', 'general_shared_cache_miss_core1']
o = [f'core{j}_L{i}_cahce_miss' for j in [0,1] for i in [1,2,3]]
p = p + o

In [55]:
env2obs = Env_observation2_subset(p)

In [57]:
r = np.array([2])

In [37]:
T = np.array(H.memory_tab)

In [43]:
def total_diversity(T):
    out = 0
    vec = []
    for j in range(T.shape[1]):
        min_ = T[:,j].min()
        max_ = T[:,j].max()
        T_normed = list((T[:,j] - min_)/(max_-min_))
        #T_normed = list(T[:,j])
        d = compute_area_interval_tree(T_normed,.001)
        out +=d
        vec.append(d)
    return out,vec

In [44]:
total_diversity(T)

(3.5592470862470837,
 [0.005999999999999893,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.003999999999999891,
  0.005999999999999893,
  0.005999999999999893,
  0.005999999999999893,
  0.05799999999999994,
  0.03799999999999992,
  0.04399999999999993,
  0.041999999999999926,
  0.19,
  0.08999999999999997,
  0.08999999999999997,
  0.03599999999999992,
  0.019999999999999907,
  0.02399999999999991,
  0.02399999999999991,
  0.03399999999999992,
  0.02399999999999991,
  0.02399999999999991,
  0.02399999999999991,
  0.30800000000000016,
  0.30800000000000016,
  0.2280000000000001,
  0.2360000000000001,
  0.0

In [46]:
p = {'core0':[H.memory_program['core0'][0]],
     'core1':[H.memory_program['core1'][0]]}

In [5]:
class OptimizationPolicykNN_(OptimizationPolicykNN):
    def __init__(self,
                k=4,
                mutation_rate = 0.1,
                max_len=50,
                num_addr = 20,
                num_bank = 4,
                min_instr = 5,
                max_instr = 50):
                super(OptimizationPolicykNN_,self).__init__(
                k=4,
                mutation_rate = 0.1,
                max_len=50,
                num_addr = 20,
                num_bank = 4,
                min_instr = 5,
                max_instr = 50)
    def select_closest_codes(self,H,coords:np.ndarray,goal:np.ndarray):
        idx = self.feature2closest_code(coords.reshape((1,-1)),goal)
        output = {"program": {"core0":[],"core1":[]},}
        for id_ in idx:
            output["program"]["core0"].append(H.memory_program["core0"][id_])
            output["program"]["core1"].append(H.memory_program["core1"][id_])
        return output
    def __call__(self,goal:np.ndarray,H:History,coords:np.ndarray)->dict:
        closest_codes = self.select_closest_codes(H,coords,goal) #most promising sample from the history
        output = self.mix(closest_codes) #expansion strategie: small random mutation
        return output
class Normalize:
    """Affine normalization
    """
    def __init__(self):
        self.min_=None
        self.max_= None
        self.g = 1
    def fit(self,x):
        self.min_ = x.min(axis=0)
        self.max_ = x.max(axis=0)
        self.g=0
    def transform(self,x):
        if self.g: 
            raise TypeError(f"User must call method Normalize.fit before calling method Normalize.transform")
        return (x - self.min_)/(self.max_-self.min_)
class IMGEP_:
    """
    N: int. The experimental budget
    N_init: int. Number of experiments at random
    H: History. Buffer containing codes and signature pairs
    G: GoalGenerator.
    Pi: OptimizationPolicy.
    """
    def __init__(self,
                 N:int,
                 N_init:int,
                 E:Env,
                 H:History,
                 G:GoalGenerator,
                 Pi:OptimizationPolicykNN,
                 periode:int = 1,
                 max_len:int = 100):
        self.N = N
        self.env = E
        self.H = H
        self.G = G
        self.N_init = N_init
        self.Pi = Pi
        self.periode = periode
        self.max_len = max_len
        self.start = 0
        self.periode_expl = 10
        self.k = 0
    def take(self,sample:dict,N_init:int):
        """Takes the ``N_init`` first steps from the ``sample`` dictionnary to initialize the exploration.
        Then the iterator i is set to N_init directly
        """
        print("sampl", sample.keys())
        for key in sample["memory_perf"].keys():
            self.H.memory_perf[key]= list(sample["memory_perf"][key][:N_init])
        self.H.memory_program["core0"] = sample["memory_program"]["core0"][:N_init]
        self.H.memory_program["core1"] = sample["memory_program"]["core1"][:N_init]
        self.start = N_init
        for j in range(len(H)):
            
            dd = {key:sample["memory_perf"][key][j] for key in sample["memory_perf"].keys() if key in pp}
            H.memory_tab.append(env_observation2_subset(dd))


class IMGEP(IMGEP_):
    def __init__(self,
                 N:int,
                 N_init:int,
                 E:Env,
                 H:History,
                 G:GoalGenerator, 
                 Pi:OptimizationPolicykNN_,
                 Norm:Normalize,
                 periode:int = 1,
                 max_len:int = 100,
                 tol = 1e-6):
        super(IMGEP,self).__init__(N,N_init, En,H,G,Pi, periode = periode, max_len = max_len)
        self.norm = Norm
        self.tol = tol
    def _SVD(self):
        in_ = np.array(np.array(self.H.memory_tab))
        self.norm.fit(in_)
        in_0 = self.norm.transform(in_)
        U,sigma,Vh =np.linalg.svd(in_0)
        return U,sigma,Vh,in_0
    def reward(self,coords):
        total_div = 0
        for j in range(coords.shape[1]):   
            compnt = coords[:,j]
            min_ = compnt.min()
            max_ = compnt.max()
            compnt_normalized = list((compnt-min_)/(max_-min_))
            total_div += compute_area_interval_tree(compnt_normalized,.001)
        reward = total_div - self.total_div
        self.total_div = total_div
        return reward
    def __call__(self):
        """Performs the exploration.
        """
        for i in range(self.start,self.N+1):
            if i%100==0:
                print(f"{i} iterations")
            if i<self.N_init:
                parameter = make_random_paire_list_instr(self.max_len,num_addr=self.env.num_addr)
            else:
                if (i-self.N_init)%(self.periode*10)==0:
                    U,sigma,Vh,in_0 = self._SVD()
                if (i-self.N_init)%self.periode==0:
                    idx_axis = np.random.randint(0,sum(sigma>self.tol))
                if i ==self.N_init:
                    in_ = np.array(self.H.memory_tab)
                    coords = self.norm.transform(in_)@Vh.transpose()
                goal = self.G(coords[:,idx_axis])
                parameter = self.Pi(goal,self.H, coords[:,idx_axis])
            observation = self.env(parameter)
            self.H.store({"program":parameter}|observation)
            self.H.memory_tab.append(env_observation2_subset(observation))
            if i >self.N_init:
                in_ = np.array(self.H.memory_tab)
                coords = self.norm.transform(in_)@Vh.transpose()
            if i>=self.N_init:
                #reward = self.reward(coords)
                pass

In [47]:
env_observation2_subset(imgep.env(p)).shape

(88,)